In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# Define parameters
# concA = 1
# concB = 2
# k1 = 3.0
# k2 = 0.6
# k3 = 0.25
# k4 = 2.95
# vol = 30 #100.0

concA = 10
concB = 20
k1 = 6
k2 = 1.0
k3 = 230
k4 = 1000
vol = 8

In [ ]:
# Simulation setup
timesteps = 10
tstride = 1
dt = 0.0001 #0.1

# Network setup
DOMAIN = [0, 500]   

# network settings
WIDTH = 40 #50 #40 #20 #15 #35 #20              # number of neurons per hidden layer
DEPTH = 15 #30 #15 #10 #10 #10 #6               # number of hidden layers
ACTIVATION = tf.nn.relu #tf.sigmoid #hidden layer activation function

# training settings
INIT_L_RATE  = 1e-3 #2e-1
FINAL_L_RATE = 1e-5 #1e-3
batch_size   = 64 #2048
data_size = 2560 #10240
data_multiplier = 5
num_batches = int(data_size*data_multiplier/batch_size)
num_epochs = 200

# Other
OUTPUT_RES = 50 #125
outscale = (DOMAIN[1] - DOMAIN[0])/OUTPUT_RES
FILENAME = "data/schlogl_data_v" + str(vol) + "_" + str(data_size) + ".txt"

In [ ]:
# Define CME birth/death rates
def lambdan(n):
    return concA*k1*n*(n-1)/vol + concB*k3*vol
def mun(n):
    return k2*n*(n-1)*(n-2)/vol**2 + n*k4

# Define intensity functions for tau-leaping
def lambda1(n):
    return concA*k1*n*(n-1)/vol
def lambda2(n):
    return k2*n*(n-1)*(n-2)/vol**2
def lambda3(n):
    return concB*k3*vol
def lambda4(n):
    return  n*k4

def ODE_func(x,k1,k2,k3,k4,a,b):
    return k1*a*x**2 - k2*x**3- k4*x + k3*b

def steadystate_solution(n):
    result = 1.0
    for i in range(n):
        result = result*(lambdan(i)/mun(i+1))
    return result
    
def terminalCondition(n):
    hist = np.linspace(DOMAIN[0], DOMAIN[1], OUTPUT_RES)
    dx = hist[1] - hist[0]
    #result = []
    for i in range(len(n)):
        index = np.where((n[i] >= hist) & (n[i] < hist + dx ))[0][0]
        iresult = np.zeros(OUTPUT_RES)
        iresult[index] = 1.0
#         if index == 0:
#             iresult[index] = 0.9
#             iresult[index+1] = 0.1
#         elif index == len(n):
#             iresult[index-1] = 0.1
#             iresult[index] = 0.9
#         else:
#             iresult[index-1] = 0.1
#             iresult[index] = 0.8
#             iresult[index+1] = 0.1
        #result.append(iresult)
    return iresult

In [ ]:
# Generate network structure

# Inputs and targets placeholders for trained data
networkInput = tf.placeholder(dtype=tf.float32, shape=(None,2), name='input')
networkTarget = tf.placeholder(dtype=tf.float32, shape=(None, OUTPUT_RES), name='target')

# Hidden layers
hidden = (DEPTH-1)*[None]
hidden[0] = networkInput
#next_layer = networkInput
for l in range(DEPTH-2):
    #without skip connections
    hidden[l+1] = tf.layers.dense(hidden[l], WIDTH, activation=ACTIVATION)
    
# Add predition outermost layer
networkPrediction = tf.layers.dense(hidden[DEPTH-2], OUTPUT_RES, activation=None, name='prediction')

In [ ]:
#def u(t, x):
#     u = networkPrediction(tf.concat([t,x],1), weights, biases)
#     return u

In [ ]:
# Training procedure

# Define loss function and error
loss = tf.reduce_mean(tf.square(networkPrediction - networkTarget))
error = tf.reduce_max(tf.abs(networkPrediction - networkTarget))

# use stochastic gradient descent with ADAM during optimization
step = tf.train.AdamOptimizer(INIT_L_RATE).minimize(loss)

# # Or use decaying learning rate with SGD
# NUM_ITER = num_batches*batch_size*num_epochs
# global_step = tf.Variable(0, trainable=False)
# learning_rate = tf.train.exponential_decay(
#     INIT_L_RATE,
#     global_step,
#     1,
#     np.exp(np.log(FINAL_L_RATE/INIT_L_RATE) / NUM_ITER),
#     staircase=True
# )
# step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [ ]:
# Print all parameters
print('\n----------------------------------------------------') 
print(' RUNNING EXPERIMENT WITH THE FOLLOWING PARAMETERS: ')
print('----------------------------------------------------\n')
print('depth:\t\t\t{}'.format(DEPTH))
print('width:\t\t\t{}'.format(WIDTH))
print('number of neurons:\t{}'.format(2+(DEPTH-2)*WIDTH))
print('number of connections:\t{}'.format(1+(DEPTH-2)*WIDTH*2+WIDTH*WIDTH*(DEPTH-3)*(DEPTH-2)//2))
print('activation:\t\t{}'.format(ACTIVATION.__name__))
print('learning rate:\t\t{} to {}'.format(INIT_L_RATE, FINAL_L_RATE))
print('epochs:\t\t{}'.format(num_epochs))
print('batch size:\t\t{}'.format(batch_size))
print('\n\n')

In [ ]:
# Generate Input/target data in parallel
generatedata = False
tfinal = 0.1*1000

# Do tau-leap integration loop (only for scalar initial condition)
def target_func_one(X0):
    N = np.zeros(timesteps)
    N[0] = X0
    for i in range(timesteps-1):
        numReactions1 = np.random.poisson(lambda1(N[i])*dt, 1)
        numReactions2 = np.random.poisson(lambda2(N[i])*dt, 1)
        numReactions3 = np.random.poisson(lambda3(N[i])*dt, 1)
        numReactions4 = np.random.poisson(lambda4(N[i])*dt, 1)
        N[i+1] = N[i] + numReactions1 - numReactions2 + numReactions3 - numReactions4
        if N[i+1] < 0:
            N[i+1] = 0
    return N

def gillespie(X0):
    N = np.zeros(1)
    N[0] = X0
    t = 0.0
    i = 0
    while t <= tfinal:
        r1 = np.random.rand()
        r2 = np.random.rand()
        rates = [lambda1(N[i]), lambda2(N[i]), lambda3(N[i]), lambda4(N[i])]
        lambda0 = np.sum(rates)
        ratescumsum = np.cumsum(rates)
        # Gillespie, time and transition
        lagtime = np.log(1.0/r1)/lambda0
        state = int(sum(r2*lambda0>ratescumsum)) + 1
        if state == 1 or state == 3:
            nextN = N[i] + 1
            N = np.append(N, nextN)
        else: # state == 2 or state == 4:
            nextN = N[i] - 1
            N = np.append(N, nextN)
        t = t + lagtime
        i = i + 1
        print(len(N))
    return N


stride = 10
x0 = np.float32(np.random.randint( DOMAIN[0], DOMAIN[1], data_size ))
def propagate(x):
    xt = np.float32(target_func_one(x))
    y0 = np.zeros(int(timesteps/stride))
    #xt = np.float32(gillespie(x))
    #y0 = np.zeros(int(len(xt)/stride))
    for j in range(len(y0)):
        y0[j] = 1.0*xt[j*stride]
    return y0
    
if generatedata:  
    num_cores = multiprocessing.cpu_count() 
    results = Parallel(n_jobs=num_cores, verbose = 2)(delayed(propagate)(i) for i in x0)
    print("Writing to file ...", end="\r")
    f = open(FILENAME,"w")
    for i in range(len(results)):
        f.write(" ".join(str(x) for x in results[i]) + "\n")
    f.close()
    print("Percentage finished:", 100, "%    ", end="\r")

In [ ]:
# readGenerateddata = True
# #FILENAME = "data/schlogl_data_v8_5120.txt"
# targetData = [None]*int(timesteps/tstride)
# if readGenerateddata:
#     data = np.genfromtxt(FILENAME, delimiter=' ')
#     inputData = data[:,0]
#     for i in range(int(timesteps/tstride)):
#         targetData[i] = terminalCondition(data[:,i])   

In [ ]:
# Read and extend data for time dependent case
readGenerateddata = True
#targetData = [None]*int(timesteps/tstride)
targetData = [None]*(data_size*data_multiplier)
if readGenerateddata:
    data = np.genfromtxt(FILENAME, delimiter=' ')
    inputData = data[:,0]
    inputData = np.array([inputData]*data_multiplier).flatten()
    tstep = np.random.randint(0, int(timesteps/tstride), data_size*data_multiplier)
    inputData = np.reshape(np.column_stack((inputData,tstep)), [-1,2])
    for i in range(data_size*data_multiplier):
        j = i%data_size
        targetData[i] = terminalCondition(np.array([data[j,tstep[i]]]))   

In [ ]:
# tstep = np.random.randint(0, int(timesteps/tstride), batch_size) #dt*j*tstride*np.ones(len(inputx))
# currentStepsData = np.take(targetData, tstep, axis=0)
# matrix = np.take(currentStepsData, indices, axis = 1)
# print(np.array(targetData).shape,currentStepsData.shape,matrix.shape)
# indices =  (indicesList[jbatch]).astype(int)
# inputBatch = np.take(inputData, indices, axis=0)

In [ ]:
# Start Tensorflow session and initialize all network variables
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
# Run the training

# run gradient descent steps with Adam
print('\nStarted training...')
print('{:8s}\t{:8s}\t{:8s}\t{:8s}'.format('Epoch', 'Iteration', 'l2-loss', 'linf-err'))
print('{:8s}\t{:8s}\t{:8s}\t{:8s}'.format(*(4*[8*'-'])))
for epoch in range(num_epochs):
    # generate random batch of inputs and corresponding target values
    indicesList = np.random.choice(data_size*data_multiplier, [num_batches, batch_size], replace = False)
    # Loop over all possible batches within the dataset
    for jbatch in range(num_batches):
        indices =  (indicesList[jbatch]).astype(int)
        inputBatch = np.take(inputData, indices, axis=0)
        #tstep = np.random.randint(0, int(timesteps/tstride), batch_size) #dt*j*tstride*np.ones(len(inputx))
        #time = tstep #dt*tstep*tstride
        #inputBatch = np.reshape(np.column_stack((inputx,time)), [-1,2])
        #inputBatch = np.reshape(inputx, [-1,1])
        
        #currentStepsData = np.take(targetData, tstep, axis=0)
        #matrix = np.take(currentStepsData, indices, axis = 1)
        #targetBatch = np.diagonal(matrix, axis1 = 0, axis2 = 1).transpose()
        #targetBatch = terminalCondition(inputx)
        targetBatch = np.take(np.array(targetData), indices, axis = 0)
        
        
        # take gradient descent step and compute loss & error
        loss_val, error_val, _ = session.run(
            [loss, error, step],
            feed_dict={networkInput: inputBatch, networkTarget: targetBatch}
        )
#         # Loop over all timesteps used for training
#         for tstep in range(int(timesteps/tstride)):
#             time = tstep*np.ones(batch_size) #dt*tstep*tstride*np.ones(BATCH_SIZE)
#             inputBatch = np.reshape(np.column_stack((inputx,time)), [-1,2])
#             targetBatch = np.take(np.array(targetData[tstep]), indices, axis = 0)

#             # take gradient descent step and compute loss & error
#             loss_val, error_val, _ = session.run(
#                 [loss, error, step],
#                 feed_dict={networkInput: inputBatch, networkTarget: targetBatch}
#             )
        if (epoch*num_batches + jbatch) % 500 == 0:
            print('{:8d}\t{:8d}\t{:1.2e}\t{:1.2e}'.format(epoch, epoch*num_batches + jbatch, loss_val, error_val))
print('...finished training.\n')

# tstep = np.random.randint(0, int(timesteps/tstride), BATCH_SIZE) #dt*j*tstride*np.ones(len(inputx))
# currentStepsData = np.take(targetData, tstep, axis=0)
# matrix = np.take(currentStepsData, indices, axis = 1)
# targetBatch = np.diagonal(matrix, axis1 = 0, axis2 = 1).transpose()

In [ ]:
# Plot for vector case

plt.rcParams['figure.figsize'] = (11,8)
# generate full sample grid of input domain
RESOLUTION = 500
xgrid = np.linspace(DOMAIN[0] + 1, DOMAIN[1], num=RESOLUTION)
#xgrid = xgrid.astype(int)
time = 10.0*np.ones(len(xgrid))
input_test_batch = np.reshape(np.column_stack((xgrid,time)) , [-1,2])
#input_test_batch = np.reshape(xgrid, [-1,1])


# get model predictions
prediction_test_batch = session.run( networkPrediction, feed_dict={networkInput: input_test_batch})

# Remove negative entries and renormalize
x0index = 250
#renormalized_output = np.mean(prediction_test_batch, axis=0)
renormalized_output = prediction_test_batch[x0index]
renormalized_output[renormalized_output<0] = 0.0
renormalized_output = renormalized_output/np.sum(renormalized_output)

# Calculate steady state analytically
n=np.linspace(DOMAIN[0], (DOMAIN[1]-DOMAIN[0])-1, DOMAIN[1]-DOMAIN[0])
ss_solution=np.zeros(len(n))
for i in range(len(n)):
    ss_solution[i] = steadystate_solution(i)

ss_solution = ss_solution/np.sum(ss_solution) 

# plot resultiung histogram
#plt.bar(np.arange(OUTPUT_RES),np.mean(prediction_test_batch, axis=0))
plt.bar(np.arange(outscale/2,outscale*OUTPUT_RES,outscale),renormalized_output/outscale, 
        width=outscale, label="NN", color=(0.0, 0.4, 1.0, 0.5))

#Plot analytic solution
plt.plot(n,ss_solution, '-r', lw = 3, label="Steady state (exact)")
#plt.plot(n,np.log(ss_solution))

#plt.ylim([0.0,0.05])
#plt.xlim([0, outscale*OUTPUT_RES])
plt.xlim([0, 400])
plt.ylim([0.0,0.02])


plt.ylabel('Probability', fontsize = 35)
plt.xlabel('$X[T]$', fontsize = 35)
plt.legend(fontsize = 35)
plt.tick_params(labelsize=30)
plt.locator_params(axis='y', nbins=6)


plt.show()

In [ ]:
tstep = np.random.randint(0, int(timesteps/tstride), BATCH_SIZE)
currentStepsData = np.take(targetData, tstep, axis=0)
matrix = np.take(currentStepsData, indices, axis = 1)
targetBatch = np.diagonal(matrix, axis1 = 0, axis2 = 1).transpose()
targetBatch[2]

In [ ]:
#Renormalize
renormalized_output = 1.0*prediction_test_batch
for i in range(RESOLUTION):
    out = prediction_test_batch[i]
    out[out<0] = 0.0
    out = out/np.sum(out)
    renormalized_output[i] = 1.0*out
    

def interactive_output(x):
    x0 = np.int(x/outscale)
    plt.bar(np.arange(1,outscale*OUTPUT_RES,outscale),renormalized_output[x0]/outscale, width=outscale)
    plt.plot(n,ss_solution, '-r')
    plt.xlim([DOMAIN[0], DOMAIN[1]])
    plt.ylim([0, 0.05])
    plt.ylabel('Histogram')
    plt.show()

interactive_plot = interactive(interactive_output, x=(DOMAIN[0],DOMAIN[1],10))
output = interactive_plot.children[-1]
interactive_plot

## TESTS

In [ ]:
len([None]*int(timesteps/tstride))

In [ ]:
plt.plot(np.mean(targetData, axis=0))

In [ ]:
x0 = 20
A = 1
B = 1
kk1 = 3*x0
kk2 = 1.0
kk3 = x0**3 - 200*x0
kk4 = (3*x0**2-200)
xmax = 100
res = 200
xx = np.linspace(-xmax,xmax,res)
y = ODE_func(xx,kk1,kk2,kk3,kk4,A,B)
plt.plot(xx,y)
plt.ylim([-5000,5000])
plt.xlim([0,50])
plt.plot(xx,0*y,'--k')

In [ ]:
print(kk1,kk2,kk3,kk4)

In [ ]:
4000/20

In [ ]:
x0 = 15
A = 10
B = 20
kk1 = 6
kk2 = 1.0
kk3 = 250
kk4 = 1000
xmax = 100
res = 200
xx = np.linspace(-xmax,xmax,res)
y = ODE_func(xx,kk1,kk2,kk3,kk4,A,B)
plt.plot(xx,y)
plt.ylim([-5000,5000])
plt.xlim([0,50])
plt.plot(xx,0*y,'--k')

In [ ]:
1875/75

In [ ]:
print(kk1,kk2,kk3,kk4)

In [ ]:
xmax = 100
x0 = 50.0
xx = np.linspace(-xmax,xmax,res)
plt.ylim([-5000,5000])
plt.plot(xx,-(xx-x0)**3+(xx-x0))

In [ ]:
numReactions1 = np.random.poisson(lambda1(20)*dt, 1)
numReactions2 = np.random.poisson(lambda2(20)*dt, 1)
numReactions3 = np.random.poisson(lambda3(20)*dt, 1)
numReactions4 = np.random.poisson(lambda4(20)*dt, 1)

In [ ]:
print(numReactions1, numReactions2, numReactions3, numReactions4)

In [ ]:
prediction_test_batch[0]

In [ ]:
n=np.linspace(0,199,200)
ss_solution=np.zeros(200)
for i in range(200):
    ss_solution[i] = steadystate_solution(i)

ss_solution = ss_solution/np.sum(ss_solution) 
plt.xlim([0,200])
#plt.plot(n,np.log(ss_solution))
plt.plot(n,ss_solution)

In [ ]:
N = OUTPUT_RES
plt.bar(np.arange(N)*outscale,np.mean(targetData,axis=0),width=outscale)
# Calculate steady state analytically
n=np.linspace(DOMAIN[0], (DOMAIN[1]-DOMAIN[0])-1, N)
ss_solution=np.zeros(N)
for i in range(N):
    ss_solution[i] = steadystate_solution(int(n[i]))

ss_solution = ss_solution/np.sum(ss_solution) 
plt.plot(n,ss_solution, '-r')
plt.show()

In [ ]:
# Define parameters
concA = 1
concB = 2
k1 = 3.0
k2 = 0.6
k3 = 0.25
k4 = 2.95
vol = 30 #100.0
def func(x):
    return k1*concA*x**2 - k2*x**3 -k4*x + k3*concB

In [ ]:
x = np.linspace(0,50,500)
plt.plot(x,func(x))

In [ ]:
xx=np.float32(np.random.randint( DOMAIN[0], DOMAIN[1], datasize ))

In [ ]:
xx[1]